# [Advent of Code 2021](https://adventofcode.com/2021)

# The toolbox


Generalised pieces of code that either can be used in multiple questions or that simply makes understand the implementation easier.

In [124]:
from collections import Counter, namedtuple
from itertools import chain
import operator


def Input(day, parser=str.strip, whole_file=False):
    "Fetch the data input from disk."
    filename = f"../data/advent2021/input{day}.txt"
    with open(filename) as fin:
        if whole_file:
            return parser(fin)
        return mapt(parser, fin)


def mapt(fn, *args):
    "Do a map, and convert the results to a tuple"
    return tuple(map(fn, *args))


avg = lambda n: sum(n) / len(n)

## [Day 1](https://adventofcode.com/2021/day/1)

In [25]:
def count_increasing_measurements(scans):
    return sum(scan > scans[index - 1] for index, scan in enumerate(scans[1:], 1))

In [26]:
data1 = Input(1, int)
count_increasing_measurements(data1)

1121

In [27]:
assert _ == 1121, 'Day 1.1'

In [28]:
chunks = [data1[i:i + 3] for i in range(len(data1) - 2)]
count_increasing_measurements(mapt(sum, chunks))

1065

In [29]:
assert _ == 1065, 'Day 1.2'

## [Day 2](https://adventofcode.com/2021/day/2)

In [30]:
def follow_commands(commands):
    x = d = 0
    for direction, amount in commands:
        if direction == "forward":
            x += amount
        elif direction == "down":
            d += amount
        elif direction == "up":
            d -= amount
        else:
            print("unknown command!")
    return x * d


def parse_input_2(line):
    chunks = line.split(" ")
    return chunks[0], int(chunks[1])


data2 = Input(2, parse_input_2)
follow_commands(data2)

2036120

In [31]:
assert _ == 2036120, "Day 2.1"

In [32]:
def follow_commands(commands):
    x = d = aim = 0
    for direction, amount in commands:
        if direction == "forward":
            x += amount
            d += aim * amount
        elif direction == "down":
            aim += amount
        elif direction == "up":
            aim -= amount
        else:
            print("unknown command!")
    return x * d


follow_commands(data2)

2015547716

In [33]:
assert _ == 2015547716, "Day 2.2"

## [Day 3](https://adventofcode.com/2021/day/3)

Not the prettiest, but a nice example of using `zip` to unzip the string into individual digits.

In [137]:
def find_power_consumption(report: list[list[int]]) -> int:
    digits = list(zip(*report))
    avg_bits = [round(avg(digits[bit_index])) for bit_index in range(len(digits))]

    most_common = lambda index: str(avg_bits[index])
    least_common = lambda index: str(int(not avg_bits[index]))

    gamma = "".join(mapt(most_common, range(len(digits))))
    epsilon = "".join(mapt(least_common, range(len(digits))))
    return int(gamma, 2) * int(epsilon, 2)


def parse_input(line):
    return mapt(int, line.strip())


data3 = Input(3, parse_input)
find_power_consumption(data3)

3969000

In [35]:
assert _ == 3969000, 'Day 3.1'

Part two was fun, here we continue to abuse bools and recurse through `bit_criteria` to remove digits.

In [36]:
def bit_criteria(
    report: list[list[int]], keep_most_common: bool, bit_index: int = 0
) -> int:
    if len(report) == 1:
        return report[0]

    digits = list(zip(*report))
    avg_bit = avg(digits[bit_index])

    # I can't be bothered to use Decimal ROUND UP here...
    most_common = 1 if avg_bit == 0.5 else round(avg_bit)

    winning_bit = most_common if keep_most_common else int(not most_common)

    return bit_criteria(
        [line for line in report if line[bit_index] == winning_bit],
        keep_most_common,
        bit_index + 1,
    )


def find_life_support_rating(report: list[list[int]]) -> int:
    oxygen_generator = bit_criteria(report[:], True)
    co2_scrubber = bit_criteria(report[:], False)

    to_int = lambda bits: int("".join(mapt(str, bits)), 2)

    return to_int(oxygen_generator) * to_int(co2_scrubber)


assert find_life_support_rating(mapt(parse_input, test_data)) == 230
find_life_support_rating(data3)

4267809

In [37]:
assert _ == 4267809, 'Day 3.2'

## [Day 4](https://adventofcode.com/2021/day/4)

Not particularly clean, and I don't really think all the sets are required, I could have simply read out rows and cols from a larger list! Oh well, did the job

In [138]:
BingoBoard = namedtuple("BingoBoard", "rows,cols")


def play_bingo(numbers: [int], boards: [BingoBoard], win_last: bool = False) -> int:
    # translate boards to a list of sets so we can easily remove seen numbers
    board_nums = dict(
        (boardId, [*[set(r) for r in board.rows], *[set(c) for c in board.cols]])
        for boardId, board in enumerate(boards)
    )

    # keep track of the most recent winningi board
    winning_board = None

    # track boards that are still in play
    active_boards = set(board_nums.keys())

    seen_numbers = set()
    for number in numbers:
        seen_numbers.add(number)

        for board_id in list(active_boards):
            for row_or_col_nums in board_nums[board_id]:
                row_or_col_nums -= seen_numbers
                if len(row_or_col_nums) == 0:
                    # Don't exit now as we'll need to remove this number
                    # from the row or col too!
                    winning_board = board_id

                    if board_id in active_boards:
                        active_boards.remove(board_id)

        have_a_winner = not win_last and winning_board is not None
        should_last_win = win_last and len(active_boards) == 0
        if have_a_winner or should_last_win:
            break

    if winning_board:
        remaining = set()
        for nums in board_nums[winning_board]:
            remaining |= nums
        return sum(remaining) * number

    print("No winning board found!")
    return -1


def parse_input(lines: [str]) -> [[int], [BingoBoard]]:
    numbers = []
    boards = []

    for line in lines:
        if not numbers:
            numbers = mapt(int, line.strip().split(","))
            continue

        if not line.strip():
            # add a new board
            boards.append(BingoBoard([], []))
            continue
        boards[-1].rows.append(mapt(int, line.strip().split()))

    # rotate rows into cols
    for board in boards:
        board.cols.extend((list(zip(*board.rows))))

    return (numbers, boards)


data4 = Input(4, parse_input, whole_file=True)
play_bingo(*data4)

8580

In [139]:
assert _ == 8580, "Day 4.1"

In [72]:
play_bingo(*data4, win_last=True)

9576

In [67]:
assert _ == 9576, "Day 4.2"

## [Day 5](https://adventofcode.com/2021/day/5)

Slight trick on this one was to sort the inputs so we knew we'd always be _increasing_ in x or y, makes finding the delta much simpler.

In [140]:
Point = namedtuple("Point", "x,y")


def count_overlapping_points(point_lines, horizontal_only=True):
    area = Counter()

    if horizontal_only:
        point_lines = [
            line
            for line in point_lines
            if line[0].x == line[1].x or line[0].y == line[1].y
        ]

    for start, stop in point_lines:
        if start.x == stop.x:
            delta = Point(0, 1)
        elif start.y == stop.y:
            delta = Point(1, 0)
        elif start.y < stop.y:
            delta = Point(1, 1)
        else:
            delta = Point(1, -1)

        line = [start]
        while line[-1] != stop:
            last_point = line[-1]
            next_point = Point(last_point.x + delta.x, last_point.y + delta.y)
            line.append(next_point)

        area.update(line)

    return len([val for _, val in area.items() if val >= 2])


def parse_input(line):
    chunks = line.split(" -> ")
    points = (
        Point(*mapt(int, chunks[0].split(","))),
        Point(*mapt(int, chunks[1].split(","))),
    )

    # sorting the points ensure we start from the left most one first
    return sorted(points)


data5 = Input(5, parse_input)
count_overlapping_points(data5)

6710

In [141]:
assert _ == 6710, 'Day 5.1'

In [135]:
count_overlapping_points(data5, horizontal_only=False)

20121

In [134]:
assert _ == 20121, 'Day 5.2'